In [ ]:
using Random

pwd()

In [2]:
struct Instance
    name::String
    graphe::BitMatrix # Matrice d'adjacence du graphe
    k::Int
end

mutable struct Solution
    nodecolors::Vector{Int} # coloration
    conflictmatrix::Matrix{Int} # matrice de conflits si l'on assigne la couleur i au nœud j
    obj::Int
end

Solution(instance::Instance,nodecolors::Vector{Int}) = Solution(nodecolors,zeros(Int,(instance.k,length(instance))),length(instance)^2)

"""Renvoie le nombre de sommets du graphe de l'instance."""
function Base.length(instance::Instance)
    return size(instance.graphe)[1]
end

function Base.length(solution::Solution)
    return length(solution.nodecolors)
end

In [3]:
function recup_numbre_dans_string(str::String,i::Int) #cette fonction recupere le nombre qui commence à l'indice i dans le string str
    j = i
    while length(str) >= j+1 && str[j+1] >= '0' && str[j+1] <= '9' #verifie si on manipule bien un chiffre
        j+=1
    end
    return (parse(Int64,str[i:j]),j)
end

"""Construit une instance de k-coloration à partir du fichier."""
function read_instance(path::String,k::Int)
    io = open(path)
    b = true
    graphe = BitMatrix
    while (b)
        ligne = readline(io)
        if length(ligne) >= 1 && ligne[1] == 'p'
            n = recup_numbre_dans_string(ligne,8)[1] #nombre de sommets dans le graphe
            graphe = falses(n,n)
        end
        if length(ligne) >= 1 && ligne[1] == 'e'
            u,j = recup_numbre_dans_string(ligne,3)
            v = recup_numbre_dans_string(ligne,j+1)[1]
            graphe[u,v] = true
            graphe[v,u] = true
        end
        if length(ligne) < 1
            b = false
        end
    end
    name = split(path,"/")[2]
    return Instance(name,graphe,k)
end

read_instance

In [4]:
# Définit les instances à évaluer
const instance_list = (
    read_instance("graphs/flat300_26_0.col", 26),
    read_instance("graphs/le450_15c.col", 15),
    read_instance("graphs/dsjc125.1.col", 5),
    read_instance("graphs/dsjc125.9.col", 44),
    read_instance("graphs/dsjc250.1.col", 8),
    read_instance("graphs/dsjc250.9.col", 72),
    read_instance("graphs/dsjc250.5.col", 28),
    read_instance("graphs/dsjc1000.5.col", 86),
    read_instance("graphs/dsjc1000.5.col", 85),
    read_instance("graphs/dsjc1000.5.col", 84)
)

(Instance("flat300_26_0.col", Bool[0 1 … 0 0; 1 0 … 1 1; … ; 0 1 … 0 1; 0 1 … 1 0], 26), Instance("le450_15c.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 0; 0 0 … 0 0], 15), Instance("dsjc125.1.col", Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], 5), Instance("dsjc125.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 0; 1 1 … 0 0], 44), Instance("dsjc250.1.col", Bool[0 0 … 0 0; 0 0 … 1 0; … ; 0 1 … 0 1; 0 0 … 1 0], 8), Instance("dsjc250.9.col", Bool[0 1 … 1 1; 1 0 … 0 1; … ; 1 0 … 0 1; 1 1 … 1 0], 72), Instance("dsjc250.5.col", Bool[0 1 … 1 0; 1 0 … 1 0; … ; 1 1 … 0 0; 0 0 … 0 0], 28), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 86), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 85), Instance("dsjc1000.5.col", Bool[0 0 … 0 0; 0 0 … 1 1; … ; 0 1 … 0 0; 0 1 … 0 0], 84))

In [5]:
"""Calcule le nombre de collisions de la coloration."""
function nbr_collision(instance::Instance,solution::Solution)
    compteur = 0 
    for j = 1:length(instance)
        for i = j+1:length(instance)
            compteur += instance.graphe[i,j] && solution.nodecolors[i] == solution.nodecolors[j]
        end
    end
    solution.obj = compteur
    return compteur
end

nbr_collision

In [6]:
"""
Calcule le nombre de collisions au sommet position en lui attribuant couleur.
"""
function nb_collision_sommet_couleur(instance::Instance,solution::Solution,position::Int,couleur::Int)
    n_collision = 0
    for  i = 1:length(solution)
        n_collision += instance.graphe[i,position] && solution.nodecolors[i] == couleur
    end
    return n_collision
end

nb_collision_sommet_couleur

In [7]:
"""Calcule l'ordre dans lequel traiter les sommets dans l'heuristique gloutonne (degré décroissant)."""
function calcul_de_ordre_des_sommets(instance::Instance) 
    deg = vec(sum(instance.graphe,dims=1))
    return sortperm(deg,rev=true)
end

"""Détermine la meilleure couleur à attribuer au nœud."""
function meilleure_couleur_locale(instance::Instance,solution::Solution,position::Int)
    best_c = 1
    best_nb_collision = length(solution)
    for c = 2:instance.k # Les couleurs sont entre 1 et k
        nb_collision = nb_collision_sommet_couleur(instance,solution,position,c)
        if nb_collision < best_nb_collision
            best_c = c
            best_nb_collision = nb_collision
        end
    end
    return best_c
end

meilleure_couleur_locale

# 1) Heuristique gloutonne

L'algorithme glouton choisi:
 - assigne successivement une couleur à chaque nœud
 - les nœuds sont traités dans l'ordre de degré décroissant
 - en choisissant pour chaque nœud la couleur minimisant le nombre de conflits avec ses voisins.

In [8]:
"""Heuristique gloutonne générant une solution par coloration successive des sommets."""
function glouton(instance::Instance)
    solution = Solution(instance,zeros(Int,length(instance))) # la couleur 0 veut dire non colorée
    ordre_des_sommets = calcul_de_ordre_des_sommets(instance)
    for i ∈ ordre_des_sommets
        c = meilleure_couleur_locale(instance,solution,i)
        solution.nodecolors[i] = c
    end
    return solution
end

glouton

In [9]:
"""Génère une solution aléatoire."""
function sol_alea(instance::Instance)
    return Solution(instance,rand(1:instance.k,length(instance)))
end

sol_alea

In [10]:
begin
    instance = instance_list[3]
    aleatoire = sol_alea(instance)

    println(instance.name,"\tk=",instance.k)
    println("SOLUTION ALÉATOIRE:")
    println(aleatoire.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,aleatoire))
    gloutonne = glouton(instance)
    println("\nSOLUTION GLOUTONNE:")
    println(gloutonne.nodecolors)
    print("collisions: ")
    println(nbr_collision(instance,gloutonne))
end

dsjc125.1.col	k=5
SOLUTION ALÉATOIRE:


[5, 2, 1, 4, 5, 1, 4, 5, 3, 1, 2, 1, 2, 4, 4, 5, 5, 4, 2, 5, 5, 5, 4, 5, 4, 2, 3, 2, 2, 1, 3, 2, 5, 5, 3, 4, 3, 2, 1, 4, 2, 5, 2, 2, 3, 1, 3, 2, 4, 2, 5, 3, 3, 4, 5, 1, 2, 4, 1, 4, 1, 5, 3, 2, 4, 2, 5, 2, 5, 1, 1, 1, 2, 2, 1, 2, 1, 4, 4, 3, 4, 3, 1, 2, 5, 5, 5, 5, 2, 1, 3, 2, 4, 5, 3, 1, 2, 4, 3, 5, 3, 3, 3, 2, 3, 3, 5, 4, 4, 1, 4, 5, 3, 1, 3, 5, 5, 4, 5, 3, 2, 5, 4, 1, 3]
collisions: 150


SOLUTION GLOUTONNE:
[3, 2, 2, 4, 2, 5, 4, 3, 4, 3, 2, 5, 3, 3, 2, 4, 4, 4, 3, 5, 5, 5, 5, 4, 5, 2, 3, 3, 2, 5, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 5, 4, 2, 2, 5, 4, 3, 5, 3, 2, 2, 2, 4, 3, 4, 4, 4, 5, 5, 2, 2, 2, 5, 3, 3, 5, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 5, 4, 4, 4, 3, 4, 2, 2, 2, 5, 3, 3, 3, 2, 4, 3, 3, 5, 5, 4, 2, 5, 3, 2, 3, 4, 5, 2, 5, 5, 3, 4, 4, 3, 2, 2, 5, 2, 4, 5, 5, 2, 4, 4, 3, 2, 2, 3]
collisions: 47


## Tests numériques de l'heuristique gloutonne

In [11]:
begin
    nsamples = 10
    println("INSTANCE NAME AND k\tMIN CONFL\tMEAN CONFL\tMAX CONFL\tTOTAL TIME\tTIME BEST SOL\tSOLS PER SECOND")
    for instance ∈ instance_list
        conflicts_samples = Vector{Int}(undef,nsamples)
        time_samples = zeros(nsamples)
        for i = 1:nsamples
            solution = glouton(instance)
            start_time = time_ns()
            conflicts = nbr_collision(instance,solution)
            run_time = time_ns()-start_time
            conflicts_samples[i] = conflicts
            time_samples[i] = run_time*1e-9
        end
        println(
            instance.name," k=",instance.k,"\t",
            minimum(conflicts_samples),"\t",
            sum(conflicts_samples)/nsamples,"\t",
            maximum(conflicts_samples),"\t",
            sum(time_samples),"\t",
            time_samples[argmin(conflicts_samples)],"\t",
            nsamples/sum(time_samples)
        ) # solutions per second irrelevant, isn't it? (since there is no local search loop)
    end
end

INSTANCE NAME AND k	MIN CONFL	MEAN CONFL	MAX CONFL	TOTAL TIME	TIME BEST SOL	SOLS PER SECOND


flat300_26_0.col k=26	226	226.0	226	0.0038031000000000002	0.0003841	2629.4338828850146
le450_15c.col k=15	362	362.0	362	0.0047627	0.0005645	2099.649358557121
dsjc125.1.col k=5	47	47.0	47	0.00018750000000000003	2.7300000000000003e-5	53333.33333333332


dsjc125.9.col k=44	21	21.0	21	0.00028490000000000004	3.49e-5	35100.03510003509
dsjc250.1.col k=8	88	88.0	88	0.0009356	0.00010850000000000001	10688.328345446773
dsjc250.9.col k=72	61	61.0	61	0.0011935	0.000125	8378.718056137412
dsjc250.5.col k=28	85	85.0	85	0.0028148000000000005	0.0002672	3552.6502771067208


dsjc1000.5.col k=86	378	378.0	378	0.0513582	0.0046594	194.71087382345954


dsjc1000.5.col k=85	398	398.0	398	0.0557786	0.010911800000000001	179.28022574966027


dsjc1000.5.col k=84	412	412.0	412	0.0443278	0.0042603	225.5920663782096


# 2) Structure de voisinage

On choisit d'utiliser un opérateur de voisinage simple:
 - Appliquer la couleur $i$ au nœud $j$ choisis de manière à minimiser le nombre de conflits.
   - On choisit un couple $(i,j)$ au hasard parmi tous les couples minimisant le nombre de conflits.
   - On tente également d'éviter que la nouvelle couleur soit la même que l'ancienne si possible.
   - Pour cela, on maintient une matrice des conflits stockant pour chaque nœud $j$ et chaque couleur $i$ le nombre de conflits en ce nœud $j$ si l'on lui applique la couleur $i$.

On obtient donc une structure de voisinage qui contient toutes les solutions dont un unique nœud diffère par sa couleur de la solution d'origine.

In [12]:
function fill_conflicts(instance::Instance,solution::Solution)
    nbr_collision(instance,solution)
    for i = 1:length(instance)
        for color = 1:instance.k
            solution.conflictmatrix[color,i] = nb_collision_sommet_couleur(instance,solution,i,color) 
        end
    end
end

function update_conflicts(instance::Instance,solution::Solution,node::Int,color::Int)
    for othernode = 1:length(instance)
        if instance.graphe[othernode,node]
            for othercolor in 1:instance.k
                solution.conflictmatrix[othercolor,othernode] += (othercolor==color) - (othercolor==solution.nodecolors[node])
            end
        end
    end
    solution.obj += solution.conflictmatrix[color,node] - solution.conflictmatrix[solution.nodecolors[node],node]
    solution.nodecolors[node] = color
end

"""Opérateur appliquant une couleur à un nœud de manière à minimiser les conflits."""
function simple_neighbor(instance::Instance,solution::Solution)
    if iszero(solution.conflictmatrix)
        fill_conflicts(instance,solution)
    end
    possible_changes = findall(
        solution.conflictmatrix .== minimum(solution.conflictmatrix) .&&
        1:instance.k .!= permutedims(solution.nodecolors)) # Éviter de garder la même solution si possible
    if isempty(possible_changes)
        return
    end
    color,node = Tuple(rand(possible_changes)) # Modification aléatoire
    update_conflicts(instance,solution,node,color)
end

simple_neighbor

In [13]:
# Test de descente locale simple
begin
    instance = instance_list[3]
    solution = sol_alea(instance)

    for t = 1:25
        simple_neighbor(instance,solution)
        println("t: ",t,"\t","conflits: ",solution.obj)
    end
end

t: 1	conflits: 145
t: 2	conflits: 142
t: 3	conflits: 141
t: 4	conflits: 138
t: 5	conflits: 135
t: 6	conflits: 132
t: 7	conflits: 128
t: 8	conflits: 128
t: 9	conflits: 126
t: 10	conflits: 123
t: 11	conflits: 123
t: 12	conflits: 121
t: 13	conflits: 118
t: 14	conflits: 117
t: 15	conflits: 114
t: 16	conflits: 112
t: 17	conflits: 111
t: 18	conflits: 109
t: 19	conflits: 108
t: 20	conflits: 107
t: 21	conflits: 104
t: 22	conflits: 103
t: 23	conflits: 102
t: 24	conflits: 98
t: 25	conflits: 98
